In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install cleantext

In [ ]:
pip install ktrain

In [ ]:
import pandas as pd
import ktrain
from ktrain import text
import cleantext
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# read training data
df = pd.read_csv("../input/nlp-getting-started/train.csv")
display(df.head())
display(df.shape)

In [ ]:
# clean training text
l=len(df)
display(l)
cleanlist=[]
textlength=[]
for i in range(l):
    ct=cleantext.clean(df.iloc[i,3], clean_all= True)
    cleanlist.append(ct)
    lct=len(ct)
    textlength.append(lct)
    

In [ ]:
# combine clean text with training data
df_clean=pd.DataFrame(cleanlist)
df_clean.columns=['cleantext']
frames=[df,df_clean]
newdf=pd.concat(frames, axis=1)
display(newdf)

In [ ]:
# distribution of clean text length
display(pd.Series(textlength).describe())

In [ ]:
#https://github.com/amaiya/ktrain
# train model
(x_train, y_train), (x_test, y_test), preproc=text.texts_from_df(newdf, 'cleantext',label_columns=['target'],
                                                                 maxlen=127,max_features=100000,
                                                                 preprocess_mode='bert', val_pct=.1)

In [ ]:
model=text.text_classifier('bert', (x_train, y_train), preproc=preproc)
learner=ktrain.get_learner(model, train_data=(x_train, y_train),
                           val_data=(x_test, y_test),
                           batch_size=32)

In [ ]:
learner.fit_onecycle(2e-5, 3)
predictor=ktrain.get_predictor(learner.model, preproc)

In [ ]:
# example
predictor.predict(['calm','earthquake'])

In [ ]:
# apply model to test data
df1 = pd.read_csv("../input/nlp-getting-started/test.csv")
display(df1.head())
display(df1.shape)

In [ ]:
# example
df1.iloc[0,3]

In [ ]:
l=len(df1)
display(l)
predlist=[]
#l=1
for i in range(l):
    ct=cleantext.clean(df1.iloc[i,3], clean_all= True)
    new=predictor.predict(ct)
    predlist.append(new)

In [ ]:
df_pred=pd.DataFrame(predlist)
df_pred.columns=['target']
frames=[df1,df_pred]
df2=pd.concat(frames, axis=1)
display(df2.head())

In [ ]:
df2.loc[df2['target']=='target','target']=1
df2.loc[df2['target']=='not_target','target']=0
display(df2['target'].mean())
df2=df2[['id','target']]
display(df2.shape)
display(df2.head())

In [ ]:
df2.to_csv("submission.csv", index=False)